# Download data from Hubble
We will use the astroquery module to query the MAST api to find HST images for CANDELS objects

In [82]:
from astroquery.mast import Observations
from astropy.io import fits
from astropy.table import Table
from astropy import units as u
from astropy import coordinates as coords
import os
import shutil

In [85]:
# Read in fits table
GZ_catalog = fits.open('schawinski_GZ_2010_catalogue.fits')
catalog = Table(GZ_catalog[1].data)
# The data is stored as individual arrays where each element represents a different galaxy
# We will pull the arrays for the OBJID and Category
Redshift = catalog['REDSHIFT'][0]
ObjID = catalog['OBJID'][0]
Morphology = catalog['GZ1_MORPHOLOGY'][0]
BPT_class = catalog['BPT_CLASS'][0]
RA = catalog['RA'][0]
DEC = catalog['DEC'][0]

for gal_ct in range(len(ObjID)):
    id_ = ObjID[gal_ct]
    redshift_ = Redshift[gal_ct]
    morph_ = Morphology[gal_ct]
    bpt_ = BPT_class[gal_ct]
    ra_ = RA[gal_ct]
    dec_ = DEC[gal_ct]

    pos = coords.SkyCoord(ra=ra_*u.degree, dec= dec_*u.degree, frame='fk5')
    obs = Observations.query_criteria(project='HST',
                                       filters='F160W',
                                       instrument_name='WFC3/IR',
                                       dataproduct_type=["image"],
                                       coordinates=pos)
    if obs:
        print("RA:%s DEC:%s"%(ra_,dec_))
        for ob in obs:
            print('    '+ob['obsid']+' - '+ob['filters'])
            
            data_products = Observations.get_product_list(ob['obsid'])#, filters=['F814W'])
            manifest = Observations.download_products(data_products, download_dir='HST/'+ob['obsid']+'-'+ob['filters'], productType='SCIENCE', extension='fits', productSubGroupDescription='DRZ')
            #Observations.download_products(ob['obsid'],
            #                    productType="SCIENCE",
            #                    curl_flag=True)
        

RA:187.24380843 DEC:-0.71369079
    2008146304 - F160W
    2008146298 - F160W
RA:224.51223292 DEC:-0.29179222
    2007927826 - F160W
INFO: Found cached file HST/2007927826-F160W/mastDownload/HST/idpw01010/idpw01010_drz.fits with expected size 12732480. [astroquery.query]
RA:217.29079067 DEC:-0.351682
    2002969515 - F160W
INFO: Found cached file HST/2002969515-F160W/mastDownload/HST/ic8601l9q/ic8601l9q_drz.fits with expected size 12608640. [astroquery.query]
RA:204.52687232 DEC:0.02051768
    2003850974 - F160W
INFO: Found cached file HST/2003850974-F160W/mastDownload/HLA/hst_12930_01_wfc3_ir_f160w/hst_12930_01_wfc3_ir_f160w_drz.fits with expected size 64805760. [astroquery.query]
INFO: Found cached file HST/2003850974-F160W/mastDownload/HLA/hst_12930_01_wfc3_ir_f160w_01/hst_12930_01_wfc3_ir_f160w_01_drz.fits with expected size 16194240. [astroquery.query]
INFO: Found cached file HST/2003850974-F160W/mastDownload/HLA/hst_12930_01_wfc3_ir_f160w_02/hst_12930_01_wfc3_ir_f160w_02_drz.fits

KeyboardInterrupt: 

In [ ]:
for root, dirs, files in os.walk("HST"):
    path = root.split(os.sep)
    #print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        if file.endswith('.fits'):
            print(os.path.join(root, file))
            shutil.copy(os.path.join(root, file), os.path.join('HST-Final', file))